In [1]:
from googleapiclient.discovery import build

import base64

from auth import authenticate

In [2]:
creds = authenticate()
service = build("gmail", "v1", credentials=creds)

In [3]:
labels = service.users().labels().list(userId="me").execute()

for label in labels['labels']:
    print(f"{label['name']} : {label['id']}")

CHAT : CHAT
SENT : SENT
INBOX : INBOX
IMPORTANT : IMPORTANT
TRASH : TRASH
DRAFT : DRAFT
SPAM : SPAM
CATEGORY_FORUMS : CATEGORY_FORUMS
CATEGORY_UPDATES : CATEGORY_UPDATES
CATEGORY_PERSONAL : CATEGORY_PERSONAL
CATEGORY_PROMOTIONS : CATEGORY_PROMOTIONS
CATEGORY_SOCIAL : CATEGORY_SOCIAL
YELLOW_STAR : YELLOW_STAR
STARRED : STARRED
UNREAD : UNREAD
1 Newsletter : Label_1936717127619429722
3 Accounts : Label_2
2 Solaranlage : Label_231893078114603930
0 anstehend : Label_4
1 Newsletter/DAV : Label_5854881578468626105
1 Newsletter/ADFC : Label_6489537413340341089
1 Newsletter/Sustainability by numbers : Label_7363869815753517727
Abnahme Mary & Abe : Label_8744948545988280091


In [4]:
ids = ( service.users().messages()
    .list(userId="me", labelIds=["Label_231893078114603930"]).execute() )
ids = [msg['id'] for msg in ids['messages']]

print(f"{len(ids)} messages found")

11 messages found


In [5]:
# https://stackoverflow.com/questions/25832631/download-attachments-from-gmail-using-gmail-api

def DownloadAttachments(service, msg_id):
    """Get and store attachment from Message with given id. """
    message = service.users().messages().get(userId="me", id=msg_id).execute()

    for part in message['payload']['parts']:
        if part['filename']:
            if 'data' in part['body']:
                data = part['body']['data']
            else:
                att_id = part['body']['attachmentId']
                att = service.users().messages().attachments().get(userId="me", messageId=msg_id,id=att_id).execute()
                data = att['data']
            file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
            path = './data/' + part['filename']

            with open(path, 'wb') as f:
                f.write(file_data)

In [8]:
for id in ids:
# id = ids[-1]

    DownloadAttachments(service, id)
# service.users().messages().trash(userId="me", id=id).execute()